In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import evaluate_predictions_from_folders
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:

folder_paths = [f'extracted_new/results/cot_random_samples_cohere_temp0/llama3']

In [4]:
df = evaluate_predictions_from_folders(folder_paths, True, 'original_label', 'extraction_complete' )
df.sort_values(by='file')

/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,file,accuracy,f1_score,recall,precision
0,20_newsgroups_llama3_cot_random_samples_cohere_llama3_random42_80_temp0_exp1.csv,0.743871,0.712652,0.708923,0.736670
5,ag_news_llama3_cot_random_samples_cohere_llama3_random42_80_temp0_exp1.csv,0.872826,0.871303,0.872826,0.876663
3,go_emotions_llama3_cot_random_samples_cohere_llama3_random42_96_temp0_exp1.csv,0.359649,0.243299,0.233286,0.292538
1,social_media_llama3_cot_random_samples_cohere_llama3_random42_60_temp0_exp1.csv,0.524103,0.340908,0.359710,0.417460
2,sst5_llama3_cot_random_samples_cohere_llama3_random42_60_temp0_exp1.csv,0.588298,0.589592,0.588298,0.594459
4,yahoo_llama3_cot_random_samples_cohere_llama3_random42_60_temp0_exp1.csv,0.738298,0.736097,0.738298,0.746844


In [5]:
pd.set_option('display.max_colwidth', 50)


In [6]:
import pandas as pd

def find_conflicting_labels(paths):
    """
    Wczytuje wiele plików i znajduje przykłady, gdzie dla tej samej wartości 'text'
    różnią się wartości w kolumnie 'extracted_label'.
    
    :param paths: lista ścieżek do plików CSV
    :return: DataFrame z przypadkami, gdzie 'extracted_label' się różni
    """
    dfs = []
    for i, path in enumerate(paths):
        df = pd.read_csv(path)
        df = df[['text', 'extracted_label']].copy()
        df = df.rename(columns={'extracted_label': f'extracted_label_{i}'})
        dfs.append(df)
    
    # Łączenie po 'text'
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = pd.merge(merged_df, df, on='text', how='inner')
    
    # Filtr: co najmniej dwie różne wartości wśród extracted_label_*
    label_cols = [col for col in merged_df.columns if col.startswith('extracted_label')]
    merged_df['n_unique'] = merged_df[label_cols].nunique(axis=1)
    
    conflicts = merged_df[merged_df['n_unique'] > 1].drop(columns='n_unique')
    
    return conflicts


In [7]:
paths = ['results/cot_random_samples_temp0.3/gpt4omini/ag_news_gpt4omini_cot_random_samples_gpt4omini_random42_80_temp0.3_exp1.csv', 
         'results/cot_random_samples_temp0.3/gpt4omini/ag_news_gpt4omini_cot_random_samples_gpt4omini_random42_80_temp0.3_exp2.csv',
         'results/cot_random_samples_temp0.3/gpt4omini/ag_news_gpt4omini_cot_random_samples_gpt4omini_random42_80_temp0.3_exp3.csv']

In [8]:
conflicts = find_conflicting_labels(paths)
conflicts

,text,extracted_label_0,extracted_label_1,extracted_label_2
2,Officials Blame Contractors in Tunnel Leak The...,Sci/Tech,Business,Sci/Tech
10,Storage On A Chip Silicon Image #39;s new lowe...,Business,Business,Sci/Tech
32,"To tithe, make it first item in budget WASHING...",World,World,Business
51,Burial Site Found Near Ancient Olympia (AP) AP...,World,Sports,World
59,Microsoft tweaks volume-licensing program Amon...,Business,Sci/Tech,Sci/Tech
89,NHLPA Disputes Claims The NHLPA said on Thursd...,Sports,Business,Sports
92,Video game giants roll out products Nintendo C...,Sci/Tech,Sci/Tech,Business
96,Briefly: Piracy crackdown yields \$2.2 million...,Sci/Tech,Sci/Tech,Business
126,Jury rules 9/11 was two attacks Attaching vict...,Business,Business,World
145,Word From on High: Jam Cell Calls Four churche...,Sci/Tech,Sci/Tech,World
